In [1]:
"""
https://pytorch-lightning.readthedocs.io/en/latest/notebooks/lightning_examples/mnist-hello-world.html

"""
import os
import torch
from pytorch_lightning import LightningModule, Trainer
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchmetrics import Accuracy
from torchvision import transforms
from torchvision.datasets import MNIST

PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 256 if AVAIL_GPUS else 64

In [2]:
class MNISTModel(LightningModule):
    def __init__(self):
        super().__init__()
        self.l1=torch.nn.Linear(28*28,10)
    def forward(self,x):
        return torch.relu(self.l1(x.view(x.size(0),-1)))
    def training_step(self,batch,batch_nb):
        x,y=batch
        loss=F.cross_entropy(self(x),y)
        return loss
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(),lr=0.02)

In [ ]:

train_ds=MNIST(PATH_DATASETS,train=True,download=T,transform=transforms.ToTensor())
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE)

mnist_model=MNISTModel()
trainer = Trainer(
    gpus=AVAIL_GPUS,
    max_epochs=3,
    progress_bar_refresh_rate=20,
)

trainer.fit(mnist_model, train_loader)

In [10]:

class LitMNIST(LightningModule):
    def __init__(self, data_dir=PATH_DATASETS, hidden_size=64, learning_rate=2e-4):
        super().__init__()

        self.data_dir=data_dir
        self.hidden_size=hidden_size
        self.learning_rate=learning_rate

        self.num_classes=10
        self.dims=(1,28,28)
        channels, width, height=self.dims
        self.transform=transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize((0.1307,),(0.3081,)),
            ]
        )
        self.model=nn.Sequential(
            nn.Flatten(),
            nn.Linear(channels*width*height,hidden_size),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_size, self.num_classes),
        )
        self.accuracy = Accuracy()
    def forward(self,x):
        x=self.model(x)
        return F.log_softmax(x,dim=1)
    def training_step(self, batch, batch_idx):
        x,y=batch
        logits=self(x)
        loss=F.nll_loss(logits,y)
        return loss

    def validation_step(self,batch,batch_idx):
        x,y=batch
        logits=self(x)
        loss=F.nll_loss(logits,y)
        preds=torch.argmax(logits,dim=1)
        self.accuracy(preds,y)

    # Calling self.log will surface up scalars for you in TensorBoard
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", self.accuracy, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        # Here we just reuse the validation_step for testing
        return self.validation_step(batch, batch_idx)

    def configure_optimizers(self):
        optimizer=torch.optim.Adam(self.parameters(),lr=self.learning_rate)
        return optimizer
    

    # DATA RELATED HOOKS

    def prepare_data(self):
        # download data
        MNIST(self.data_dir,train=True,download=True)
        MNIST(self.data_dir,train=False,download=True)
    
    def setup(self,stage=None):
        # Assign train/val datasets for use in dataloaders
        if stage == "fit" or stage is None:
            mnist_full = MNIST(self.data_dir, train=True, transform=self.transform)
            self.mnist_train, self.mnist_val = random_split(mnist_full, [55000, 5000])

        # Assign test dataset for use in dataloader(s)
        if stage == "test" or stage is None:
            self.mnist_test = MNIST(self.data_dir, train=False, transform=self.transform)
    
    def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size=BATCH_SIZE)

    def val_dataloader(self):
        return DataLoader(self.mnist_val, batch_size=BATCH_SIZE)

    def test_dataloader(self):
        return DataLoader(self.mnist_test, batch_size=BATCH_SIZE)



In [11]:
model=LitMNIST()
trainer = Trainer(
    gpus=AVAIL_GPUS,
    max_epochs=3,
    progress_bar_refresh_rate=20,
)

trainer.fit(model)

/Users/jungdonghyuk/repositories/ml_study/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:90: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=20)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name     | Type       | Params
----------------------------------------
0 | model    | Sequential | 55.1 K
1 | accuracy | Accuracy   | 0     
----------------------------------------
55.1 K    Trainable params
0         Non-trainable params
55.1 K    Total params
0.220     Total estimated model params size (MB)


Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/Users/jungdonghyuk/repositories/ml_study/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/Users/jungdonghyuk/repositories/ml_study/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 2: 100%|██████████| 939/939 [00:08<00:00, 112.00it/s, loss=0.216, v_num=1, val_loss=0.168, val_acc=0.950]


In [12]:
trainer.test()

/Users/jungdonghyuk/repositories/ml_study/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1398: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `test(ckpt_path='best')` to use and best model checkpoint and avoid this warning or `ckpt_path=trainer.checkpoint_callback.last_model_path` to use the last model.
  rank_zero_warn(
Restoring states from the checkpoint path at /Users/jungdonghyuk/repositories/ml_study/MNIST/lightning_logs/version_1/checkpoints/epoch=2-step=2579.ckpt
Loaded model weights from checkpoint at /Users/jungdonghyuk/repositories/ml_study/MNIST/lightning_logs/version_1/checkpoints/epoch=2-step=2579.ckpt
/Users/jungdonghyuk/repositories/ml_study/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_

Testing: 100%|██████████| 157/157 [00:01<00:00, 125.25it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'val_acc': 0.9491000175476074, 'val_loss': 0.1691868156194687}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 157/157 [00:01<00:00, 122.44it/s]


[{'val_loss': 0.1691868156194687, 'val_acc': 0.9491000175476074}]

In [13]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/